In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [2]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'superconductivity-model'
id_name = ''
target = '81'

file_name = "superconductivity_test.csv"
test_file_name = "superconductivity_test-no-label.csv"

kaggle_file_name = ""
kaggle_test_file_name = ""
kaggle_pred_file_name = ""

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

   0            1                  2                   3                   4  \
0  5    102.03412  73.68762243902441    86.0522347667303    66.7278915783626   
1  5  62.00128064        72.82697508  44.947006650016796   55.98375541016971   
2  4    76.797975        76.15722384    68.8456154214048    71.7975646710252   
3  5     82.30211  58.54843474212029    63.3778043146984  36.334197756645395   
4  4     92.50535   60.9463657142857     68.030394691755    36.8961153538594   

                  5                   6          7                   8  \
0  1.44847054470796    1.40681582333855    155.006  28.565149756097604   
1  1.33206796353657  1.1954115695490402   128.2426         35.58553992   
2  1.27635655631019    1.16171538539942    98.2287         28.52982256   
3  1.40715386953297     1.0307238905809  124.90825  37.200619269341004   
4  1.17203527939876    1.00387418185352   134.3606    36.3205485714286   

                    9  ...    71                  72                  73  

In [5]:
f = open(test_file_name)
count = 0
pred = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    pred.append(float(response))
    count = count + 1


In [16]:
print(pred[:20])

import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred)

y_true[5339] = 92.29642486572266
ind = np.where(y_true == "critical_temp")
print(ind)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_true, y_pred)
print("test mse = {}".format(mse))

[27.093339920043945, 49.271976470947266, 13.649847984313965, 20.54791259765625, 18.9859676361084, 3.900615692138672, 63.32382583618164, 4.091673851013184, 67.1026611328125, 3.8301923274993896, 81.39025115966797, 2.521423578262329, 19.34471893310547, 36.800594329833984, 52.290470123291016, 43.195980072021484, 2.481990337371826, 84.00130462646484, 11.7262544631958, 4.635318756103516]
(array([], dtype=int64),)
test mse = 90.18084858764321


In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

FileNotFoundError: [Errno 2] File  does not exist: ''

In [14]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [5]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

NameError: name 'kaggle_pred_df' is not defined

In [22]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 07:51:25 GMT'},
  'RetryAttempts': 0}}